In [2]:
import json
import os
from datasets import Dataset, load_from_disk, concatenate_datasets, load_dataset

import torch
from torch import tensor

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/scratch/gpfs/JHA/mb5157/tokenizers/biomedbert_fast_tokenizer')

from collections import defaultdict
from pathlib import Path

/scratch/gpfs/JHA/mb5157/CONDA/envs/graphmert/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.set_option('display.max_colwidth', None)

# Draft

## Entities

In [3]:
path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/qwen3-32b_all_clean_results/'
sample = load_from_disk(path)
# sample = sample.select(range(1000, 2000))

In [4]:
sample

Dataset({
    features: ['id', 'input_ids', 'head', 'position', 'relation', 'predictions', 'tails', 'text', 'triple_similarities'],
    num_rows: 1341137
})

In [5]:
path = '/projects/JHA/shared/dataset/qwen32b/350k_abstracts_qwen3-32b_heads_all_result_glmkg'
sample = load_from_disk(path)
sample

Dataset({
    features: ['input_ids', 'leaf_node_ids', 'leaf_relationships', 'attention_mask', 'input_nodes', 'start_indices', 'special_tokens_mask', 'head_positions', 'id', 'top_k_relations_with_scores_cui'],
    num_rows: 989666
})

In [7]:
sample = pd.read_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/qwen32b_beta_0.67_deduplicated.csv')
sample

,id,text,head,relation,tail
0,413486,"to the quartiles of urinary albumin excretion ( q1 [ < 7. 7 mg / g ], 17. 1 μg / ml ; q2 [ 7. 7 – 15. 0 mg / g ], 17. 5 μg / ml ; q3 [ 15. 1 – 61. 4 mg / g ], 18. 6 μg / ml ; q4 [ ≥61. 5 mg / g ], 22. 3 μg / ml ; p for trend = 0. 003 ) under adjustment with other covariates. a positive correlation was found between plasma lrg1 level and urinary albumin excretion ( ρ = 0. 256, p < 0.",plasma lrg1 level,associated_with,urinary albumin excretion
1,450824,"the aim of this article is to assess the association of subclinical atherosclerosis with accelerated epigenetic age and to investigate the underlying mechanisms mediating this association. methods and results whole blood methylomics, transcriptomics, and plasma proteomics were obtained for 391 participants of the progression of early subclinical atherosclerosis study. epigenetic age was calculated from methylomics data for each participant. its divergence from chronological age is termed epigenetic age acceleration. subclinical atherosclerosis burden was estimated by multi - territory 2d / 3d vascular ultrasound and by coronary artery calcification. in healthy individuals, the presence, extension, and progression of subclinical atherosclerosis were associated with a significant acceleration of the grim epigenetic age, a",epigenetic age acceleration,associated_with,subclinical atherosclerosis
2,406192,"several hallmark processes of cancer. over the past decades, researchers have identified the epithelial protein lost in neoplasm ( eplin, also known as lima1 ) as a key regulator of cytoskeletal dynamics, cytoskeletal organization, motility, as well as cell growth and metabolism. dysregulation of eplin is implicated in various aspects of cancer progression, such as tumor growth, invasion, metastasis, and therapeutic resistance. its altered expression levels or activity can disrupt cytoskeletal dynamics, leading to aberrant cell motility and invasiveness characteristic of malignant cells. moreover, the involvement of eplin in cell growth and metabolism underscores its significance in orchestrating key processes essential for cancer cell survival and","epithelial protein lost in neoplasm ( eplin, also known as lima1 )",plays_role,cytoskeleton
3,512330,"and conditions have been identified as modulators of nlrp3 inflammasome activation, and at the same time, many of the nuclear receptors are known to regulate, and interact with these factors, including cellular metabolism and various signaling pathways. nuclear receptors are in the focus of many researches, as these receptors are easy to manipulate by lipid soluble molecules. importantly, nuclear receptors mediate regulatory mechanisms at multiple levels : not only at transcription level, but also in the cytosol via non - genomic effects. their importance is also reflected by the numerous approved drugs that have been developed in the past decade to specifically target nuclear receptors subtypes. researches aiming to delineate mechanisms that regulate nlrp3 inflammasome",nuclear receptors,plays_role,nlrp3 inflammasome
4,771977,"by a decay in cellular functions and proliferation, resulting in increased cellular damage and death. this condition plays an essential role in the aging process and significantly contributes to the development of age - related complications. on the other hand, ferroptosis is a systemic cell death pathway characterized by excessive iron accumulation followed by the generation of reactive oxygen species ( ros ). oxidative stress is a common trigger of this condition and may be induced by various factors such as toxins, drugs, and inflammation. ferroptosis is linked to numerous disorders, including cardiovascular disease, neurodegeneration, and cancer. senescence is believed to contribute to the decay in tissue and organ functions occurring with",senescence,associated_with,ferroptosis
...,...,...,...,...,...
109288,374992,"function, is a critical role played by the brain ’ s h

In [6]:
path = '/scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/qwen32b/350k_abstracts_qwen3-32b_heads_all'
sample = load_from_disk(path)
# sample = sample.select(range(1000, 2000))

In [7]:
sample

Dataset({
    features: ['input_ids', 'leaf_node_ids', 'leaf_relationships', 'attention_mask', 'input_nodes', 'start_indices', 'special_tokens_mask', 'head_positions', 'id'],
    num_rows: 989666
})

In [ ]:
def count_heads(batch):
    content = {"n_heads": [len(json.loads(s)) if s is not None else 0 for s in batch["head_positions"]]}
    return content

ds2 = sample.map(
    count_heads,
    batched=True,
    batch_size=10_000,
    keep_in_memory=False,  # or `load_from_cache_file=False` to force re-compute
)

# 2) pull out that list and sum
total = sum(ds2["n_heads"])
print(f"Total number of head‐entities: {total}")

In [ ]:
def output_entities_sample(dataset, start=0):
    end = start + 100
    items = dataset[start: end]
    for i, values in \
                enumerate(zip(items['input_ids'], items['head_positions'])):
        input_ids, head_positions = values
        # print(i + start)
        # print(tokenizer.decode(input_ids, skip_special_tokens=True))
        # print()
        print(head_positions)
        # print()

In [ ]:
output_entities_sample(sample, start=50_000)

## Load predictions

In [32]:
def output_prediction_sample(dataset, start=0):
    end = start + 100
    items = dataset[start: end]
    for i, values in \
                enumerate(zip(items['input_ids'], items['head'], items['relation'], items['predictions'])):
        input_ids, head, relation, predictions = values
        if relation != 'associated_with':
            print(i + start)
            print(tokenizer.decode(input_ids, skip_special_tokens=True), '\n')
            print('head:', head)
            print('relation:', relation)
            print('predictions:', predictions)
            print()

In [28]:
prefix = '/scratch/gpfs/JHA/mb5157/repos/graphmert/graphmert'
prediction_suffix = 'predictions/top_20/'

In [29]:
model_path = 'outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004'
preprocessed_dataset_path = os.path.join(prefix, model_path, prediction_suffix)

dataset_predictions_span = load_from_disk(preprocessed_dataset_path)
print(dataset_predictions_span)

Dataset({
    features: ['input_ids', 'attention_mask', 'id', 'head', 'head_len', 'position', 'relation', 'relation_num', 'predictions', 'probabilities'],
    num_rows: 1341137
})


In [8]:
output_prediction_sample(dataset_predictions_span, start=0)

0
to examine direct and indirect costs, early retirement, cardiovascular events and mortality over 5 years in people with atherosclerotic cardiovascular disease ( ascvd ) and matched controls in sweden. methods individuals aged ≥ 16 years living in sweden on 01 january 2012 were identified in an existing database. individuals with ascvd were propensity score matched to controls without ascvd by age, sex and educational status. we compared direct healthcare costs ( inpatient, outpatient and drug costs ), indirect costs ( resulting from work absence ) and the risk of stroke, myocardial infarction ( mi ) and early retirement. results after matching, there were 231, 417 individuals in each cohort. total 

head: atherosclerotic cardiovascular disease
relation: cause_of
predictions: disease cardiovascular atherosclerotic with vascular atherosclerosis ##oscler due ##vd cerebrovascular arteri chronic ischemic risk coronary of disorder dementia hypercholesterolemia ##osclerosis

2
to examine di

In [ ]:
df = dataset_predictions_span.to_pandas()
df['sequence'] = df['input_ids'].apply(lambda ids: tokenizer.decode(ids, skip_special_tokens=True))

# 3) Group & aggregate
def aggregate_group(g):
    # for this one id, build { head → [ (relation, predictions), … ] }
    out = {}
    for head, sub in g.groupby('head', sort=False):
        out[head] = list(zip(sub['relation'], sub['predictions']))
    return pd.Series({
        'sequence': g['sequence'].iloc[0],
        'results': out
    })

agg = df.groupby('id', sort=False).apply(aggregate_group).reset_index()
agg

## Token Combination

In [14]:
def output_token_comb(dataset_r1, idx_range=(0, 50)):
    start, end = idx_range
    items = dataset_r1.select(range(start, end))

    for i, zipped in \
                enumerate(zip(items['input_ids'], items['head'], items['relation'], items['predictions'], items['tails'])):
        input_ids, head, relation, predictions, tails = zipped
        print(i + idx_range[0])
        print(tokenizer.decode(input_ids, skip_special_tokens=True), '\n')
        print('head:', head)
        print('relation:', relation)
        print('predictions:', predictions)
        print('tails:', tails)
        print()

In [17]:
llm_name = 'qwen3-8b'
glm_path = Path(f'../outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004')
prediction_suffix = 'predictions/top_20/'
dataset_path = str(glm_path / prediction_suffix / f'{llm_name}_all_clean')
# dataset_path = str(glm_path / prediction_suffix)


test_dataset = load_from_disk(dataset_path)
output_token_comb(test_dataset, idx_range=(0, 50))

0
to examine direct and indirect costs, early retirement, cardiovascular events and mortality over 5 years in people with atherosclerotic cardiovascular disease ( ascvd ) and matched controls in sweden. methods individuals aged ≥ 16 years living in sweden on 01 january 2012 were identified in an existing database. individuals with ascvd were propensity score matched to controls without ascvd by age, sex and educational status. we compared direct healthcare costs ( inpatient, outpatient and drug costs ), indirect costs ( resulting from work absence ) and the risk of stroke, myocardial infarction ( mi ) and early retirement. results after matching, there were 231, 417 individuals in each cohort. total 

head: atherosclerotic cardiovascular disease
relation: cause_of
predictions: disease cardiovascular atherosclerotic with vascular atherosclerosis ##oscler due ##vd cerebrovascular arteri chronic ischemic risk coronary of disorder dementia hypercholesterolemia ##osclerosis
tails: []

1
to 

# Check novelty wrt seed KG

In [30]:
df_seed = pd.read_csv("/projects/JHA/shared/dataset/qwen32b/alpha_0.55/seed_kg_qwen32b_0.55.csv")
df_seed = df_seed.drop('Unnamed: 0', axis=1)

In [26]:
df_seed[(df_seed.relation_type == 'same_as')].head(50) # & (df_result.tail == 'demyelination')]

,head,relation_type,tail
741,microalbuminuria,same_as,urine microalbumin detected
993,atherosclerosis,same_as,arteriosclerotic vascular disease
1392,preeclampsia,same_as,severe pre - eclampsia
1510,chronic kidney disease,same_as,chronic renal insufficiency
1708,anti - thyroglobulin antibodies ( a - tg ),same_as,thyroglobulin antibody measurement
1744,dumping syndrome,same_as,postgastric surgery syndrome
1871,gestational hypertension,same_as,maternal hypertension
2149,thyroglobulin antibodies ( tgab ),same_as,thyroglobulin antibody measurement
2368,cushing ’ s syndrome,same_as,hypercortisolism
2507,hepatic steatosis,same_as,steatosis of liver


In [32]:
# 
# import re

# If you want to match the whole word “demyelination” (so that “demyelination” but not “demyelinations” or “predemyelination”), use word‐boundaries:
# mask = df_result["tail"].str.contains(r"\bdemyelination\b", regex=True)

# If you don’t care about word‐boundaries and just want the literal substring “demyelination” anywhere:
mask = df_seed["head"].str.contains("sglt2is", regex=False)

df_filtered = df_seed[mask]
print(len(df_filtered))




0


In [ ]:
df_filtered[df_filtered.relation_type == 'has_finding_site']

In [ ]:
df_seed = df_seed.rename(columns={'relation_type': 'relation'})

In [ ]:
# 5. Save the plot to a file.
output_filename = 'plots/relation_distribution_seed_kg.pdf'
fig.savefig(output_filename, format='pdf', dpi=300) 

# Gemini

In [ ]:
# prefix = '../'
# model_path = 'outputs/350abstracts/gemini/span/hgat/lim2_rel33/bs256_lr_0.0004/predictions/top_20/'
# dataset = load_from_disk(os.path.join(prefix, model_path, 'gemini-2.0_0-50000'))
# dataset

In [ ]:
def output_sample(dataset, start=0):
    end = start + 50
    items = dataset.select(range(start, end))
    for i, values in \
                enumerate(zip(items['input_ids'], items['head'], \
                              items['relation'], items['predictions'], items['tails'])):
        input_ids, head, relation, predictions, tails = values
        print(i + start)
        print(tokenizer.decode(input_ids), '\n')
        print('head:', head)
        print('relation:', relation)
        print('predictions:', predictions)
        print('tails:', tails)
        print()

In [ ]:
prefix = '../'
model_path = 'outputs/350abstracts/qwen32/score0.55/span/rel29/bs256_lr_0.0004/'
llm_name = 'qwen32b'

dataset_path = os.path.join(prefix, model_path, f'predictions/top_20')
add_hgat_dataset  = load_from_disk(dataset_path)

In [ ]:
def get_df_triples(dataset):
    df = dataset.to_pandas()
    df = df[['head', 'relation', 'tails']]
    df = df.explode('tails')
    df = df.dropna(subset=['tails'])
    df = df.rename(columns={"tails": "tail"})
    df = df.drop_duplicates(subset=['head', 'relation', 'tails'], keep='first').reset_index(drop=True)
    return df

def get_df_triples_cnt(dataset):
    df = dataset.to_pandas()
    df = df[['head', 'relation', 'tails']]
    df = df.explode('tails')
    df = df.dropna(subset=['tails'])
    df = df.rename(columns={"tails": "tail"})
    df = df.groupby(by=['head', 'relation', 'tail']).agg(total=("tail", "count")).reset_index().sort_values('total', ascending=False)
    return df

In [ ]:
alpha = '0.55'
path_template = '/projects/JHA/shared/dataset/qwen32b/alpha_{alpha}/span/cleaned_graphs/qwen32b_beta_{beta}.csv'

span67 = pd.read_csv(path_template.format(alpha=alpha, beta=0.67))
span65 = pd.read_csv(path_template.format(alpha=alpha, beta=0.65))

In [ ]:
relation_counts_span = [
    span65['relation'].value_counts(),
    df_seed['relation'].value_counts(),
]

# 2. Combine the counts into a single DataFrame for grouped plotting.
# This ensures all relations from all DataFrames are included.
all_relations = relation_counts_span[0].index.union(relation_counts_span[1].index)
comparison_df = pd.DataFrame({
    'Beta 65': relation_counts_span[0].reindex(all_relations, fill_value=0),
    'Seed KG': relation_counts_span[1].reindex(all_relations, fill_value=0),
    
})
# Sort by the relation name in alphabetical order
comparison_df = comparison_df.sort_index(ascending=True)


# 3. Create the bar chart.
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(16, 10))

# Plot the combined DataFrame to create a grouped bar chart with three colors
comparison_df.plot(kind='bar', ax=ax, color=['#1f77b4', '#ff7f0e', '#2ca02c'], width=0.8)

# 4. Customize the plot for better aesthetics and readability.
ax.set_title('Comparison of Relation Frequencies (Logarithmic Scale)', fontsize=20, pad=20)
ax.set_ylabel('Number of Occurrences (Log Scale)', fontsize=16, labelpad=15)
ax.set_xlabel("") # Explicitly remove the default x-axis label

# Set the y-axis to a logarithmic scale
ax.set_yscale('log')

# Customize the legend
ax.legend(title='Extracted KG:', fontsize=16, title_fontsize=16)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=50, ha='right', fontsize=14)
plt.yticks(fontsize=16)

# Add a light grid to the y-axis
ax.yaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.25)
ax.set_axisbelow(True)

# Remove top and right spines for a cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# 5. Ensure all elements fit into the saved image.
fig.tight_layout()

In [ ]:
# 5. Save the plot to a file.
output_filename = 'plots/relation_distribution_in_kgs.pdf'
fig.savefig(output_filename, format='pdf', dpi=300) 

### check if the result contains a keyword

In [ ]:
target_word = "DNA Methylation".lower()
# mask = span67["head"].str.contains(target_word, regex=False)

m1 = span67["tail"].str.contains(target_word, regex=False, na=False)
m2 = span67["head"].str.contains(target_word, regex=False, na=False)
mask = m1 | m2


df_filtered = span67[mask]
df_filtered = df_filtered.drop(columns='Unnamed: 0')
df_filtered.index = [''] * len(df_filtered)
print(len(df_filtered))
df_filtered[['head', 'relation', 'tail']]

In [ ]:
df_filtered[['head', 'relation', 'tail']].iloc[:50]

In [ ]:
target_word = "DNA Methylation"

mask = span67["head"].str.contains(target_word, regex=False)

# m1 = span67["tail"].str.contains(target_word, regex=False, na=False)
# m2 = span67["head"].str.contains(target_word, regex=False, na=False)
# mask = m1 | m2


df_filtered = span67[mask]
df_filtered = df_filtered.drop(columns='Unnamed: 0')
df_filtered.index = [''] * len(df_filtered)
print(len(df_filtered))
df_filtered[['head', 'relation', 'tail']]

# other

In [ ]:
llm = 'qwen32b'  # or 'gemini' for Gemini model
score = 0.55
dataset_with_injections_path = f'/scratch/gpfs/JHA/mb5157/large_data/diabetes_2025_6years/dataset/for_training/{llm}'
injections = pd.read_csv(os.path.join(dataset_with_injections_path, f'350k_injections_{llm}_score_{score}.csv'))
injections

In [ ]:
injections[injections.relation_type == 'plays_role'][['head', 'relation_type', 'tail']].head(40)

In [ ]:
# result_after.drop_duplicates(subset=['head', 'relation', 'tail'], keep='first')

In [ ]:
tmp = pd.read_csv('/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/ablations/no_hgat/cleaned_graphs/qwen32b_beta_0.67.csv')
tmp

In [ ]:
tmp[['head', 'relation', 'tail']].iloc[50: 100]

## find an example

In [20]:
path = '/projects/JHA/shared/dataset/qwen32b/alpha_0.55/span/cleaned_graphs/validated_yes_no_duplicates.csv'
tmp = pd.read_csv(path)

In [21]:
tmp[['id', 'text', 'head', 'relation', 'tail']][(tmp.relation == 'same_as') & (tmp.relation != 'plays_role')].iloc[0:50]

,id,text,head,relation,tail
1214,658925,"and deleterious mediators, but an increasing number of more recent studies show multiple beneficial properties under various pathological conditions. many of the discrepancies in the published studies are due to the investigation of different species or mixtures of lysophatidylcholines and the use of supra - physiological concentrations in the absence of serum or other carrier proteins. furthermore, interpretation of the results is complicated by the rapid metabolism of lysophosphatidylcholine ( lpc ) in cells and tissues to pro - inflammatory lysophosphatidic acid. interestingly, most of the recent studies, in contrast to older studies, found lower lpc plasma levels associated with unfavorable disease outcomes. being the most abundant lysophospholipid",lpc,same_as,lysophosphatidylcholine
2505,832507,". • adipocytokines are involved in the pathogenesis of diseases in nearly all body systems. • adipocytokines had an impact on metabolic, oncologic, and rheumatologic disorders. introduction adipose tissue secretes various bioactive peptides / proteins, immune molecules and inflammatory mediators which are known as adipokines or adipocytokines. adipokines play important roles in the maintenance of energy homeostasis, appetite, glucose and lipid metabolism, insulin sensitivity, angiogenesis, immunity and inflammation. enormous number of studies from all over the world proved that adipocytokines are involved in the pathogenesis of diseases affecting nearly all body systems, which raises the question whether we can always bla",adipocytokines,same_as,adipokines
3969,240683,"3. 34, 95 % ci 1. 20 – 9. 26 ) and all ‐ cause death ( hr 3. 10, 95 % ci 1. 72 – 5. 57 ). all these outcomes were statistically significantly different among four quartiles of hba1c ( log ‐ rank p ‐ value < 0. 05 ). fatal and non ‐ fatal stroke showed no statistically significant association with high hba1c variability. conclusion high hba1c variability in the highest quartile showed a statistically significant association with multiple adverse cardiovascular events in an asian population. minimizing hba1c fluctuation during long ‐ term follow up should be another important objective for type 2 diabetes patients. high glycated hemoglobin variability",high glycated hemoglobin variability,same_as,high a1c variability
3982,832507,". • adipocytokines are involved in the pathogenesis of diseases in nearly all body systems. • adipocytokines had an impact on metabolic, oncologic, and rheumatologic disorders. introduction adipose tissue secretes various bioactive peptides / proteins, immune molecules and inflammatory mediators which are known as adipokines or adipocytokines. adipokines play important roles in the maintenance of energy homeostasis, appetite, glucose and lipid metabolism, insulin sensitivity, angiogenesis, immunity and inflammation. enormous number of studies from all over the world proved that adipocytokines are involved in the pathogenesis of diseases affecting nearly all body systems, which raises the question whether we can always bla",adipokines,same_as,adipocytokines
4131,645166,", and 9 studies on homeostatic model assessment for insulin resistance ( homa - ir ) showed a summary md of - 3. 35 mg / dl ( 95 % ci : - 8. 10 to 1. 40, p = 0. 16 ), - 0. 21 % ( 95 % ci : - 0. 33 to - 0. 09, p = 0. 001 ), - 1. 05 µiu / ml ( 95 % ci : - 1. 53 to - 0. 58, p < 0. 001 ), and - 0. 44 ( 95 % ci : - 0. 82 to - 0. 05",homeostatic model assessment for insulin resistance,same_as,homa index
4528,284248,"with diabetes. prader – willi syndrome ( pws ) is a rare, multisystemic, genetic disorder involving the hypothalamus. it is caused by loss of expression of paternally inherited genes in chromosome 15 q11 - 13 region. the estimated incidence is around 1 in 20. 000 births. pws is characterized by a complex lifelong trajectory involving neurodevelopmental, nutritional, endocrine, metabolic, and behavioral changes. the major symptoms 